# CART
CART树可以分为回归树和决策树。GBDT中的树是CART回归树。

CART分类树算法使用**基尼系数**来代替信息增益比，**基尼系数代表了模型的不纯度**，基尼系数越小，不纯度越低，特征越好。

在分类问题中，假设有$K$ 个类别，第$k$个类别的概率为$p_k$ ,则基尼系数为：

$$Gini(P)=\sum^K_{k=1}P_k(1-P_k) = 1 - \sum^K_{k=1}P^2_k$$

<img style="float: center;" src="GBDT_pics/Cart_1.png" width=600 height=600>

## 分类树

1. **首先计算各特征的基尼指数，选择最优特征以及其最优切分点。特征记为$A_1,A_2,A_3,A_4$**

以上述表格为例，计算年龄的基尼系数，首先年龄分为三个类型，需要计算**年龄特征$A_1$下三个类别的基尼系数**

$$Gini(D,A_1 = 青年) =\frac{5}{15} (2\times \frac{2}{5}\times (1 - \frac{2}{5})) + \frac{10}{15} (2\times \frac{7}{10}\times (1 - \frac{7}{10}))$$

$\frac{5}{15}$是三个年龄类型中青年的概率，$\frac{2}{5}$与$\frac{7}{10}$是青年与非青年中正样本的比例

2. **其次计算每个特征的最优切分点，找到基尼系数最小的特征切分点。**

## 回归树

1. 原始数据集$S$，此时树的深度$depth=0$。

2. 针对集合$S$，遍历每一个$feature$的每一个$value$，用该$value$将原数据集$S$分裂成2个集合：左集合$S_{left}$
 ($<=value$的样本)、右集合$S_{right}$($>value$的样本)，每一个集合也叫做一个结点。
 
3. 分别计算这2个集合的$mse$，找到使得$(left_{mse}+right_{mse})$最小的那个$value$，记录下此时的$feature$名称和$value$，这个就是**最佳分割特征以及该特征的最佳分割值**；

每一个集合/结点$mse$的计算方法如下:

- $mean = \frac{1}{N} \sum_{i\in S_{left}} \ \ y_i , mse = \sum(y_i - mean)^2$，$mean$就是落在该叶子节点当中的预测值 


# Boosting

Boosting主要关注降低偏差，因此Boosting能基于泛化性能相当弱的学习器构建出很强的集成；Bagging主要关注降低方差，因此它在不剪枝的决策树、神经网络等学习器上效用更为明显。

## Adaboost

1. 确定样本集

2. 初始化样本数据的权重，如每个样本的权重为1/n(假设样本个数为n)

3. 进行1，2，...，T轮迭代

        a. 归一化样本权重
        b. 对样本集进行训练，并计算训练误差
        c. 选择误差最小的分类器作为本轮的分类器
        d. 根据预测结果更新样本数据的权重：预测错误样本增加权重，预测正确样本降低权重
        e. 计算分类器的权重

4. 根据分类器的预测结果及其权重加权表决最终结果

### 例

|序号|1|2|3|4|5|6|7|8|9|10|
|-|-|-|-|-|-|-|-|-|-|-|
|x|0|1|2|3|4|5|6|7|8|9|
|y|1|1|1|-1|-1|-1|1|1|1|-1|

- **第一个个体学习机**

首先认为每个$x_i$的权重一样(本例中为$\frac{1}{10}$)

1. **计算切分点**

第一个学习器的最优切分点为$2.5$，此时误差最小，即

$$G_1(x) =  \begin{cases}
1, &x<2.5\\
-1, &x>2.5
\end{cases}
$$

2.  **计算分类器权重**

第一个分类器中有三个错误，即6，7，8

计算误差$e_1 = 0.3$，计算分类器权重为$\alpha_i = \frac{1}{2}ln\frac{1-e_i}{e_i}$(这么做的原因仅当$e_i<\frac{1}{2},\alpha_i>0$，学习器才有意义)

3. **更新训练数据的权值分布**


$w_{m+1,i} = \cfrac{w_{m,i}}{Z_m}exp(-\alpha_my_iG_m(x_i)), i = 1,2,...,N$

$Z_m=w_{m,i}\ exp(-\alpha_my_iG_m(x_i))$

其中$y_i,G_m(x_i)$为真实标签与预测标签，当预测正确时为$y_iG_m(x_i) = 1$，错误时$y_iG_m(x_i) = -1$，这样在公式中可以保证分类正确的权值下降，分类错误的上升

其中$Z_m$为正则化参数，保证权值和唯一


- **第m个个体学习机**

第$m$个学习机在上一轮更新后的权值分布上重新重复计算过程

# GBDT

梯度提升决策树（Gradient Boosting Decision Tree，GBDT）是一种基于boosting集成学习思想的加法模型，训练时采用前向分布算法进行贪婪的学习，每次迭代都学习一棵CART树来拟合之前 t-1 棵树的预测结果与训练样本真实值的负梯度**(平方损失函数中负梯度是残差)**。

## GBDT回归树

与CART类似的过程，拟合变为负梯度

## GBDT分类树

对数似然函数进行拟合多分类拟合